In [ ]:
#Anotaciones sobre imagenes
#Es posible usar este software online: https://www.makesense.ai/


The TensorFlow Object Detection API is an open-source computer vision framework for building object detection and image segmentation models that can localize multiple objects in the same image.

In [ ]:
#Clonando desde el repositorio
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 81991, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 81991 (delta 293), reused 400 (delta 255), pack-reused 81533
Receiving objects: 100% (81991/81991), 596.36 MiB | 17.63 MiB/s, done.
Resolving deltas: 100% (58492/58492), done.


In [ ]:
##Insalando TF2 ODA en Colab
%%bash
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf2/setup.py .
python -m pip install --use-feature=2020-resolver .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696908 sha256=373c79439da1f4382c44ecd3b98ba4d7aa252cf0da1b374044f3d77c4fbf1636
  Stored in directory: /tmp/pip-ephem-wheel-cache-uo44otn2/wheels/2c/e0/d7/9685ee19da997d4c97d45ab09b394097be67758bc07728fa80
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf
from google.colab import files
import pathlib
import itertools
import random

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

from object_detection.utils import config_util
from object_detection.utils import colab_utils
from object_detection.builders import model_builder
%matplotlib inline

Usaremos este github para bajar un modelo de Faster RCNN pre entrenado con COCO
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wget
model_link = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz"
wget.download(model_link)


'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz'

In [ ]:
import tarfile
tar = tarfile.open('/content/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz')
tar.extractall('.') 
tar.close()

FileNotFoundError: ignored

In [ ]:
#Baje el ZIP de GUNS y descomprimalo en el content/
%cd /content/
!unzip gunniest.zip  
!unzip /content/gunniest/gunniest.zip  -d /content/gunniest

/content
Archive:  gunniest.zip
replace gunniest/armas (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: gunniest/armas (1).jpg  
  inflating: gunniest/armas (10).jpg  
  inflating: gunniest/armas (100).jpg  
  inflating: gunniest/armas (101).jpg  
  inflating: gunniest/armas (102).jpg  
  inflating: gunniest/armas (103).jpg  
  inflating: gunniest/armas (104).jpg  
  inflating: gunniest/armas (105).jpg  
  inflating: gunniest/armas (106).jpg  
  inflating: gunniest/armas (107).jpg  
  inflating: gunniest/armas (108).jpg  
  inflating: gunniest/armas (109).jpg  
  inflating: gunniest/armas (11).jpg  
  inflating: gunniest/armas (110).jpg  
  inflating: gunniest/armas (111).jpg  
  inflating: gunniest/armas (112).jpg  
  inflating: gunniest/armas (113).jpg  
  inflating: gunniest/armas (114).jpg  
  inflating: gunniest/armas (115).jpg  
  inflating: gunniest/armas (116).jpg  
  inflating: gunniest/armas (117).jpg  
  inflating: gunniest/armas (118).jpg  
  inflating: gunnies

Editando el archivo pipeline.config 

1. num_classes De 90 a 1 dado que solo detectaremos armas
2. image_resizer Las imagenes originales conque fue entrenado eran de 640x640, reducir el tamaño reduce el tiempo de entrenamiento. Vamos a reducirlas a 320x320 
3. num_steps Por defecto viene de 25000, vamos a dejarlo en 1000 para reducir el tiempo de entrenamiento. Uted puede incrementarlo si el LOSS aun esta cayendo.
4. batch_size , por defecto viene en 64, dado que usaremos colab colocaremos solo 1 (una sola imagen sera colocada en memoria mientras entrena)  
5. fine_tune_checkpoint Este apuntara a la ruta del FASTER-CNN descargado. Recuerde que no entrenaremos el modelo desde cero.
6. fine_tune_checkpoint_type a detection, dado que estamos entreando un modelo para detectar y no para clasificar.
7. train_input_reader
***the label_map_path apunta al TFRecords. 
8. eval_input_reader Igual que para  train_input_reader pero para el conjunto de TEST.

In [ ]:
%cd /content/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/
!sed -i 's/num_classes: 90/num_classes: 1/' pipeline.config
!sed -i 's/min_dimension: 640/min_dimension: 320/' pipeline.config
!sed -i 's/max_dimension: 640/max_dimension: 320/' pipeline.config
!sed -i 's/num_steps: 25000/num_steps: 1000/' pipeline.config
!sed -i 's/batch_size: 64/batch_size: 1/' pipeline.config


/content/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8


In [ ]:
!pip install annotated_images

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import annotated_images

In [ ]:
!git clone https://gist.github.com/ef9eb79c3f2f516ed1e4f793150d6f76.git